In [ ]:
import json

In [ ]:
review = open("data/yelp_academic_dataset_review.json", "r", encoding='utf-8')
business = open("data/yelp_academic_dataset_business.json", "r", encoding='utf-8')
user = open("data/yelp_academic_dataset_user.json", "r", encoding='utf-8')
checkin = open("data/yelp_academic_dataset_checkin.json", "r", encoding='utf-8')
covid19 = open("data/yelp_academic_dataset_covid_features.json", "r", encoding='utf-8')